In [1]:
import sys
sys.path.append('/workspace/code/Fake-News-Detection-Dataset')
sys.path.append('/workspace/code/Fake-News-Detection-Dataset/detection')
# print(sys.path)

In [68]:
from transformers import AutoModel, AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import T5ForConditionalGeneration, AutoTokenizer, T5TokenizerFast
import json
from glob import glob
import os
from tqdm.auto import tqdm
import nltk
import numpy as np
import argparse
from methods_search import generation
import torch
import pandas as pd
from accelerate import Accelerator
from transformers import AutoConfig
from detection.model import BERT
from kobert_transformers import get_tokenizer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset 
# finetuning transformers==4.4.2

In [71]:
class BaitDataset(Dataset):
    def __init__(self, id_list, title_list, body_list, tokenizer):
        
        self.tokenizer = tokenizer
        self.max_len = 512

        self.title_list = title_list
        self.body_list = body_list
        self.label_list = [0]*len(title_list)
        self.id_list = id_list

    def __len__(self):
        return len(self.id_list)

    def __getitem__(self, index):
        title = self.title_list[index] 
        body = self.body_list[index] 
        label = self.label_list[index]

        encoding = self.tokenizer.encode_plus( # automatically pad first
            text = title,
            text_pair = body,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        
        doc = {}
        doc['input_ids']=encoding['input_ids'].flatten()
        doc['attention_mask']=encoding['attention_mask'].flatten()

        return doc, label
    
def convert_device(inputs: dict, device: str) -> dict:
    for k in inputs.keys():
        inputs[k] = inputs[k].to(device)

    return inputs

def evaluate(model, dataloader, device: str = 'cpu'):
    correct = 0
    total = 0
    total_score = []
    total_preds = []
    total_targets = []

    model.eval()
    with torch.no_grad():
        for idx, (inputs, targets) in tqdm(enumerate(dataloader), desc = 'EVAL : ', total = len(dataloader)):
            inputs, targets = convert_device(inputs, device), targets.to(device)
            
            # predict
            outputs = model(**inputs)
            outputs = torch.nn.functional.softmax(outputs, dim=1)
            
            # total loss and acc
            preds = outputs.argmax(dim=1)

            correct += targets.eq(preds).sum().item()
            total += targets.size(0)

            total_score.extend(outputs.cpu().tolist())
            total_preds.extend(preds.cpu().tolist())
            total_targets.extend(targets.cpu().tolist())
            
    metrics = {
        'acc' : correct/total
    }


    return metrics
            

In [115]:
df = pd.read_csv('../../data/Fake/content_chunking_forward/generated/fake_top3.csv')
df = pd.read_csv('../../data/Fake/content_chunking_forward/filtered/fake_top3_90_99.csv')

### 하이퍼파라미터 실험

In [77]:
df.head(2)

,news_id,original_title,original_content,sim_news_id,fake_title,category,label,sim_news_content,sim_news_title
0,PO_M03_417681,정경두 “SLBM 도발은 남북군사합의에 없다”,정경두 국방부 장관은 2일 국회 국방위원회 국정감사에서 북한의 이날 미사일 발사가 ...,PO_M03_115891,"정경두 \""北 미사일 발사, ‘비밀적 도발' 없다\""",PO,0,북한이 순항미사일을 발사한 지 3일 만인 15일 단거리 탄도미사일 2발을 동해상으로...,"한국 첫 SLBM 쏜날, 북 탄도미사일 도발"
1,IS_M10_079706,"상반기 사이버 보안 3대 위협, 랜섬웨어 그리고…",올해 상반기 국내외 사이버 보안 위협 특징으로 랜섬웨어가 꼽혔다.\nPC와 모바일에...,IS_M10_085484,[202020202020202020202020202020202020202020202...,IS,0,최근 성인용 게임으로 위장한 악성코드가 대량으로 유포되고 있어 이용자 주의가 당부된...,"\""19금 게임인줄\"" 받고 보니 좀비PC 만드는 악성코드"


In [6]:
accelerator = Accelerator()
tokenizer = AutoTokenizer.from_pretrained('../finetuning/Models/ke-t5-base-newslike_epoch3')
model = AutoModelForSeq2SeqLM.from_pretrained('../finetuning/Models/ke-t5-base-newslike_epoch3')
model = accelerator.prepare(model)
sum_model=None,
sum_tokenizer=None

In [131]:
fake_df = df.sample(8)

prefix = "summarize: "
batch_size = 8
use_metadata = 'content'
method = 'chunking'
direction = 'forward'

generation_config = {'do_sample':True,
                     'num_beams':True,
                     'min_length':10,
                     'max_length':32,
                     'temperature':0.8,
                     'top_k':30,
                     'top_p':0.8}

In [132]:
generated_title_list = generation(
    fake_df['original_title'].to_list(),
    fake_df['original_content'].to_list(),
    fake_df['sim_news_title'].to_list(),
    fake_df['sim_news_content'].to_list(),
    prefix,
    model,
    tokenizer,
    accelerator,
    sum_model,
    sum_tokenizer,
    batch_size=batch_size,
    use_metadata=use_metadata,
    method=method,
    direction=direction,
    max_input_length=512,
    **generation_config
    )

In [133]:
for origin, sim, gen, fake in zip(fake_df['original_title'].to_list(), fake_df['sim_news_title'].to_list(), generated_title_list, fake_df['fake_title'].to_list()):
    print('원본 뉴스 제목 : ', origin)
    print('유사 뉴스 제목 : ', sim)
    # print('생성 뉴스 제목 : ', gen)
    print('가짜 뉴스 제목 : ', fake)
    print('-------'*20)

원본 뉴스 제목 :  “지금 우리가 종철이에게 하고픈 이야기 ‘노래’로 표현했죠”
유사 뉴스 제목 :  “민중가요의 시대는 갔어도 노래는 계속된다”
가짜 뉴스 제목 :  ‘박종철 추모 곡’ 첫 첫 콘서트
--------------------------------------------------------------------------------------------------------------------------------------------
원본 뉴스 제목 :  차벽은 정당하다고? 차벽 없을 때 더 평화로웠는데…
유사 뉴스 제목 :  조건부 집회 허용에 민주노총 \"생색내기 판결…예정대로 진행\"
가짜 뉴스 제목 :  \"노조노조 집회·시위 자유권 한계\"
--------------------------------------------------------------------------------------------------------------------------------------------
원본 뉴스 제목 :  \"코로나19 '집콕'에 부부관계는 멀어지고 형제애는 깊어져\"
유사 뉴스 제목 :  ‘중국 CIA’ 국가안전부 넘버2, 미국 망명설 확산
가짜 뉴스 제목 :  \"집콕으로 부부관계 악화\"
--------------------------------------------------------------------------------------------------------------------------------------------
원본 뉴스 제목 :  ‘상위 1%’의 민주주의 사랑법
유사 뉴스 제목 :  [2000 책의 흐름] '부자 아빠 가난한 아빠'
가짜 뉴스 제목 :  [이슈분석]부자아빠 가난한 아빠, 민주주의 사랑하는 사람
-----------------------------------------------------------------------------------------------------------

### 모델 output

In [112]:
bert_path = '../../results/content_chunking_forward/best_model.pt'

bert_tokenizer = get_tokenizer()

model_config = AutoConfig.from_pretrained('monologg/kobert')
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
train_model = BERT( # model.py class
    config          = model_config,
    num_classes     = 2
)
train_model.load_state_dict(torch.load(bert_path)) # load pre-trained model
train_model.to(device)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


BERT(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        

In [110]:
testset = BaitDataset(
    id_list=fake_df['news_id'].to_list(),
    title_list=generated_title_list,
    body_list=fake_df['original_content'].to_list(),
    tokenizer = bert_tokenizer
)

testloader = DataLoader(
    testset, 
    batch_size  = 8
)

baseset = BaitDataset(
    id_list=fake_df['news_id'].to_list(),
    title_list=fake_df['fake_title'].to_list(),
    body_list=fake_df['original_content'].to_list(),
    tokenizer = bert_tokenizer
)

baseloader = DataLoader(
    testset, 
    batch_size  = 8
)

test_metrics = evaluate(
    model        = train_model, 
    dataloader   = testloader, 
    device       = device,
)

base_metrics = evaluate(
    model        = train_model, 
    dataloader   = testloader, 
    device       = device,
)

print('생성 뉴스 정확도 : ', test_metrics['acc'])
print('가짜 뉴스 정확도 : ', base_metrics['acc'])


EVAL : 100%|██████████| 13/13 [00:01<00:00, 10.41it/s]

생성 뉴스 정확도 :  0.55
가짜 뉴스 정확도 :  0.55
